This script pulls team-level boxscore data from the nba.com api.  For the purposes of this project, I pulled boxscore data from the 2000-01 season through the 2021-22 season.

Credit to jnish23 for the helper functions used below.

In [1]:
from nba_api.stats.static import players, teams
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscorescoringv2

import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import time as time
from time import sleep
from IPython.core.display import clear_output
import sqlite3

In [2]:
def season_string(season):
    return str(season) + '-' + str(season+1)[-2:]

In [3]:
def add_basic_boxscores(conn, start_season, end_season, if_exists='append'):
    """This function pulls basic team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_basic_boxscores in the sqlite db"""
    
    cur = conn.cursor()
    table_name = 'team_basic_boxscores'
    
    if if_exists == 'replace':
        cur.execute('DROP TABLE IF EXISTS ' + table_name)
        cur.execute('VACUUM')
        
    cur.execute("""CREATE TABLE IF NOT EXISTS {} (SEASON TEXT, TEAM_ID INTEGER, TEAM_ABBREVIATION TEXT, 
        TEAM_NAME TEXT, GAME_ID TEXT, GAME_DATE DATE, MATCHUP TEXT, WL TEXT, MIN INTEGER, FGM INTEGER, FGA INTEGER, 
        FG_PCT FLOAT, FG3M INTEGER, FG3A  INTEGER, FG3_PCT FLOAT, FTM INTEGER, FTA INTEGER, FT_PCT FLOAT, OREB INTEGER,
        DREB INTEGER, REB INTEGER, AST INTEGER, STL INTEGER, BLK INTEGER, TOV INTEGER, PF INTEGER, PTS INTEGER, 
        PLUS_MINUS INTEGER)""".format(table_name))    
    
    for season in tqdm(range(start_season, end_season+1)):
        season_str = season_string(season)
        season_boxscores = []

        for season_type in ['Regular Season', 'Playoffs']:
            boxscores = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
            season_boxscores.append(boxscores)
            sleep(2)
        season_df = pd.concat(season_boxscores)
        season_df['SEASON'] = season_str
        season_df.drop(columns = ['SEASON_ID', 'VIDEO_AVAILABLE'], inplace=True)
        
        season_df.to_sql(table_name, conn, if_exists='append', index=False)
        
        sleep(3)
        
    #cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return None

In [4]:
def add_advanced_boxscores(conn, start_season, end_season, if_exists='append'):
    """
    This function pulls advanced team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_advanced_boxscores in the sqlite db
    
    Note: Because of timeout errors and that we have to pull each game's individually, each season takes 1-2 hours.
    If some games were not pulled in certain seasons, you can use the update functions to gather those individual games.
    """
    
    table_name = 'team_advanced_boxscores'
    game_ids_not_added = []
    
    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)
        conn.execute('VACUUM')
    
    conn.execute('''CREATE TABLE IF NOT EXISTS {} (GAME_ID TEXT, TEAM_ID INTEGER, TEAM_NAME TEXT, 
        TEAM_ABBREVIATION TEXT, TEAM_CITY TEXT, MIN TEXT, E_OFF_RATING FLOAT, OFF_RATING FLOAT, E_DEF_RATING FLOAT, 
        DEF_RATING FLOAT, E_NET_RATING FLOAT, NET_RATING FLOAT, AST_PCT FLOAT, AST_TOV FLOAT, 
        AST_RATIO FLOAT, OREB_PCT FLOAT, DREB_PCT FLOAT, REB_PCT FLOAT, E_TM_TOV_PCT FLOAT, 
        TM_TOV_PCT FLOAT, EFG_PCT FLOAT, TS_PCT FLOAT, USG_PCT FLOAT, E_USG_PCT FLOAT, E_PACE FLOAT, 
        PACE FLOAT, PACE_PER40 FLOAT, POSS FLOAT, PIE FLOAT)'''.format(table_name))
    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)
        season_team_boxscores = []

        for season_type in ['Regular Season', 'Playoffs']:
            logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
            game_ids = logs['GAME_ID'].unique()

            for i in range(0, len(game_ids), 100):
                print('games {} to {}'.format(i, i+100))
                for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                    try:
                        team_boxscores = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id).get_data_frames()[1]                    
                        team_boxscores.to_sql(table_name, conn, if_exists='append', index=False)
                    except:
                        game_ids_not_added.append(game_id)
                    sleep(2)
                sleep(120)
                clear_output(wait=True)
        
        sleep(120)
        
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    print(len(game_ids_not_added), ' games not added')
    return game_ids_not_added

In [5]:
def add_scoring_boxscores(conn, start_season, end_season, if_exists='append'):
    """
    This function pulls scoring team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_scoring_boxscores in the sqlite db.
    
    Note: Because of timeout errors and that we have to pull each game's individually, each season takes 1-2 hours.
    If some games were not pulled in certain seasons, you can use the update functions to gather those individual games.
    """
    
    table_name = 'team_scoring_boxscores'
    game_ids_not_added = []

    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)
        conn.execute('VACUUM')
    
    conn.execute('''GAME_ID TEXT, TEAM_ID INTEGER, TEAM_NAME TEXT, TEAM_ABBREVIATION TEXT, TEAM_CITY TEXT,
       MIN TEXT, PCT_FGA_2PT FLOAT, PCT_FGA_3PT FLOAT, PCT_PTS_2PT FLOAT, PCT_PTS_2PT_MR FLOAT,
       PCT_PTS_3PT FLOAT, PCT_PTS_FB FLOAT, PCT_PTS_FT FLOAT, PCT_PTS_OFF_TOV FLOAT,
       PCT_PTS_PAINT FLOAT, PCT_AST_2PM FLOAT, PCT_UAST_2PM FLOAT, PCT_AST_3PM FLOAT,
       PCT_UAST_3PM FLOAT, PCT_AST_FGM FLOAT, PCT_UAST_FGM FLOAT)'''.format(table_name))
    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)
        season_team_boxscores = []

        for season_type in ['Regular Season', 'Playoffs']:
            logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
            game_ids = logs['GAME_ID'].unique()

            for i in range(0, len(game_ids), 100):
                print('games {} to {}'.format(i, i+100))
                for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                    try:
                        scoring_boxscores = boxscorescoringv2.BoxScoreScoringV2(game_id).get_data_frames()[1]
                        scoring_boxscores.to_sql(table_name, conn, if_exists='append', index=False)
                    except:
                        game_ids_not_added.append(game_id)
                    sleep(2)
                sleep(120)
                clear_output(wait=True)
        print(season_str, ' completed')
        sleep(120)
        
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return game_ids_not_added

In [6]:
## Update basic team gamelogs and player gamelogs

def update_team_basic_boxscores(conn, season):
    table_name = 'team_basic_boxscores'
    season_str = season_string(season)
        
    dfs = []
    for season_type in ['Regular Season', 'Playoffs']:
        team_gamelogs = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
        dfs.append(team_gamelogs)
        
    team_gamelogs_updated = pd.concat(dfs)
    team_gamelogs_updated['SEASON'] = season_str
    team_gamelogs_updated.drop(columns = ['SEASON_ID', 'VIDEO_AVAILABLE'], inplace=True)
    
    team_gamelogs_updated.to_sql(table_name, conn, if_exists='append', index=False)

    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return None

In [7]:
def update_team_advanced_boxscores(conn, season, dates):
    table_name = 'team_advanced_boxscores'
    
    season_str = season_string(season)
    
    game_ids_not_added = []
    
    # Pull the GAME_IDs from my data
    game_ids_in_db = pd.read_sql('''SELECT DISTINCT team_basic_boxscores.GAME_ID FROM team_basic_boxscores
                INNER JOIN team_advanced_boxscores 
                ON team_basic_boxscores.GAME_ID = team_advanced_boxscores.GAME_ID
                AND team_basic_boxscores.TEAM_ID = team_advanced_boxscores.TEAM_ID
                WHERE SEASON = "{}" '''.format(season_str), conn)

    game_ids_in_db = game_ids_in_db['GAME_ID'].tolist()
    
    missing_game_ids = []
    if len(dates) != 0:
        for date in dates:
            gamelogs = leaguegamelog.LeagueGameLog(
                season=season_str, date_from_nullable=date, date_to_nullable=date).get_data_frames()[0]
            missing_game_ids.extend(gamelogs['GAME_ID'].unique())
            
    else:        
        # get up to date GAME_IDs
        to_date_game_ids = []
        for season_type in ['Regular Season', 'Playoffs']:
            to_date_gamelogs = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
            to_date_game_ids.extend(to_date_gamelogs['GAME_ID'].unique())

        # See which game_ids are missing
        missing_game_ids = set(to_date_game_ids) - set(game_ids_in_db)
        
    num_games_updated = len(missing_game_ids)
    print("num_games_updated:", num_games_updated)
    
    if num_games_updated == 0:
        print("All team advanced boxscores up to date in season {}".format(season_str))
        return None
    
    for game_id in tqdm(missing_game_ids, desc='progress'):
        try:
            boxscores = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id).get_data_frames()[1]
            boxscores.to_sql(table_name, conn, if_exists='append', index=False)
            sleep(2)
        except:
            game_ids_not_added.append(game_id)  
    
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT max(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return game_ids_not_added

In [8]:
def update_team_scoring_boxscores(conn, season, dates):
    table_name = 'team_scoring_boxscores'

    season_str = season_string(season)

    game_ids_not_added = []

    # Pull the GAME_IDs from my data
    game_ids_in_db = pd.read_sql(f'''SELECT DISTINCT team_scoring_boxscores.GAME_ID FROM team_basic_boxscores
                INNER JOIN team_scoring_boxscores 
                ON team_basic_boxscores.GAME_ID = team_scoring_boxscores.GAME_ID
                AND team_basic_boxscores.TEAM_ID = team_scoring_boxscores.TEAM_ID
                WHERE SEASON = "{season_str}" ''', conn)

    game_ids_in_db = game_ids_in_db['GAME_ID'].tolist()

    missing_game_ids = []
    if len(dates) != 0:
        for date in dates:
            gamelogs = leaguegamelog.LeagueGameLog(
                season=season_str, date_from_nullable=date, date_to_nullable=date).get_data_frames()[0]
            missing_game_ids.extend(gamelogs['GAME_ID'].unique())

    else:
        # get up to date GAME_IDs
        to_date_game_ids = []
        for season_type in ['Regular Season', 'Playoffs']:
            to_date_gamelogs = leaguegamelog.LeagueGameLog(
                season=season_str, season_type_all_star=season_type).get_data_frames()[0]
            to_date_game_ids.extend(to_date_gamelogs['GAME_ID'].unique())

        # See which game_ids are missing
        missing_game_ids = set(to_date_game_ids) - set(game_ids_in_db)
        
    num_games_updated = len(missing_game_ids)
    print("num_games_updated:", num_games_updated)

    if num_games_updated == 0:
        print("All team advanced boxscores up to date in season {}".format(season_str))
        return None

    for game_id in tqdm(missing_game_ids, desc='progress'):
        try:
            boxscores = boxscorescoringv2.BoxScoreScoringV2(
                game_id).get_data_frames()[1]
            boxscores.to_sql(table_name, conn,
                             if_exists='append', index=False)
            sleep(2)
        except:
            game_ids_not_added.append(game_id)

    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT max(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(
        table_name, table_name))
    conn.commit()

    return game_ids_not_added

In [9]:
#connect to sqlite database
con = sqlite3.connect('/nba.db')

In [27]:
#pull basic boxscores from the 2000-01 to 2021-22 seasons
add_basic_boxscores(con, 2000, 2022)

100%|██████████| 23/23 [03:16<00:00,  8.52s/it]


In [10]:
#check basic boxscore data
team_basic_boxscores_df = pd.read_sql('select * from team_basic_boxscores', con)
team_basic_boxscores_df.head()

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,2021-22,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19,LAL vs. GSW,L,240,45,...,5,40,45,21,7,4,18,25,114,-7
1,2021-22,1610612744,GSW,Golden State Warriors,0022100002,2021-10-19,GSW @ LAL,W,240,41,...,9,41,50,30,9,2,17,18,121,7
2,2021-22,1610612751,BKN,Brooklyn Nets,0022100001,2021-10-19,BKN @ MIL,L,240,37,...,5,39,44,19,3,9,13,17,104,-23
3,2021-22,1610612749,MIL,Milwaukee Bucks,0022100001,2021-10-19,MIL vs. BKN,W,240,48,...,13,41,54,25,8,9,8,19,127,23
4,2021-22,1610612754,IND,Indiana Pacers,0022100003,2021-10-20,IND @ CHA,L,240,42,...,8,43,51,29,2,10,17,24,122,-1


In [22]:
#pull in advanced boxscore data.  Each season took a couple hours to pull, so the seasons were pulled individually
game_ids_excluded = add_advanced_boxscores(con, 2003, 2003)

4  games not added


In [13]:
#concatenate advanced boxscore data. Since the data pull process was very slow I used three different devices to pull
#in data separately. Here, I am concatenating the advanced boxscore data from the different devices

team_advanced_boxscores_main = pd.read_csv('team_advanced_boxscores.csv')
team_advanced_boxscores_1 = pd.read_csv('team_advanced_boxscores_151109060200.csv')
team_advanced_boxscores_2 = pd.read_csv('team_advanced_boxscores_1410070501.csv')

team_advanced_boxscores = pd.concat([team_advanced_boxscores_main, team_advanced_boxscores_1, team_advanced_boxscores_2])
team_advanced_boxscores.drop(columns='Unnamed: 0', inplace=True)

In [14]:
print(len(team_advanced_boxscores))
team_advanced_boxscores.head()

56164


,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,22100002,1610612747,Lakers,LAL,Los Angeles,240:00,98.0,101.8,106.0,107.1,...,16.1,0.553,0.551,1.0,0.198,115.28,112.5,93.75,112.0,0.422
1,22100002,1610612744,Warriors,GSW,Golden State,240:00,106.0,107.1,98.0,101.8,...,15.0,0.516,0.570,1.0,0.200,115.28,112.5,93.75,113.0,0.578
2,22100001,1610612749,Bucks,MIL,Milwaukee,240:00,117.7,124.5,101.8,102.0,...,7.8,0.538,0.562,1.0,0.194,105.02,102.0,85.00,102.0,0.593
3,22100001,1610612751,Nets,BKN,Brooklyn,240:00,101.8,102.0,117.7,124.5,...,12.7,0.542,0.552,1.0,0.200,105.02,102.0,85.00,102.0,0.407
4,22100003,1610612754,Pacers,IND,Indiana,240:00,111.4,114.0,107.1,116.0,...,15.9,0.561,0.607,1.0,0.199,112.22,106.5,88.75,107.0,0.520


In [15]:
#team_advanced_boxscores['GAME_ID'] = team_advanced_boxscores['GAME_ID'].astype('string')
#team_advanced_boxscores['TEAM_ID'] = team_advanced_boxscores['TEAM_ID'].astype('string')

team_advanced_boxscores['GAME_ID'] = '00' + team_advanced_boxscores['GAME_ID'].astype('string')
#team_advanced_boxscores['TEAM_ID'] = team_advanced_boxscores['TEAM_ID'].astype('string')

In [16]:
#find missing games in advanced boxscore data
game_ids = list(team_basic_boxscores_df['GAME_ID'].unique())
advanced_boxscore_game_ids = list(team_advanced_boxscores['GAME_ID'].unique())

In [17]:
missing_game_ids = np.setdiff1d(game_ids, advanced_boxscore_game_ids)
missing_game_ids_2 = np.setdiff1d(advanced_boxscore_game_ids, game_ids)
print(len(game_ids))
print(len(advanced_boxscore_game_ids))
print(len(missing_game_ids))
print(missing_game_ids)
print(len(missing_game_ids_2))
print(missing_game_ids_2)

28154
28082
73
['0020300086' '0020300453' '0020300778' '0020300833' '0020400002'
 '0020400003' '0020400020' '0020400023' '0020400029' '0020400048'
 '0020400190' '0020400554' '0020500010' '0020500017' '0020500019'
 '0020500020' '0020500021' '0020500022' '0020500028' '0020500030'
 '0020500032' '0020500038' '0020600103' '0020600111' '0020600119'
 '0020600127' '0020600128' '0020600141' '0020600637' '0020600667'
 '0020600670' '0020600672' '0020600673' '0020600682' '0020700927'
 '0020700976' '0020701078' '0020800104' '0020800859' '0020801063'
 '0021000552' '0021001153' '0021201036' '0021201037' '0021201202'
 '0021201214' '0021300797' '0021400627' '0021500139' '0021500141'
 '0021500151' '0021600004' '0021700109' '0021701174' '0021800152'
 '0021800566' '0021800911' '0022000447' '0022100041' '0022100215'
 '0022100461' '0022100463' '0022100534' '0022100692' '0022101047'
 '0022101216' '0040800104' '0040800124' '0040800152' '0040800162'
 '0040800165' '0041700232' '0041800233']
1
['0042100405']


In [21]:
#save full, concatenated advanced boxscore data to sqlite database

con.execute('''DROP TABLE team_advanced_boxscores''')
con.execute('''CREATE TABLE IF NOT EXISTS {} (GAME_ID TEXT, TEAM_ID INTEGER, TEAM_NAME TEXT, 
        TEAM_ABBREVIATION TEXT, TEAM_CITY TEXT, MIN TEXT, E_OFF_RATING FLOAT, OFF_RATING FLOAT, E_DEF_RATING FLOAT, 
        DEF_RATING FLOAT, E_NET_RATING FLOAT, NET_RATING FLOAT, AST_PCT FLOAT, AST_TOV FLOAT, 
        AST_RATIO FLOAT, OREB_PCT FLOAT, DREB_PCT FLOAT, REB_PCT FLOAT, E_TM_TOV_PCT FLOAT, 
        TM_TOV_PCT FLOAT, EFG_PCT FLOAT, TS_PCT FLOAT, USG_PCT FLOAT, E_USG_PCT FLOAT, E_PACE FLOAT, 
        PACE FLOAT, PACE_PER40 FLOAT, POSS FLOAT, PIE FLOAT)'''.format('team_advanced_boxscores'))

team_advanced_boxscores.to_sql('team_advanced_boxscores', con, if_exists='append', index=False)
cur = con.cursor()
cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format('team_advanced_boxscores', 'team_advanced_boxscores'))
con.commit()

In [22]:
team_advanced_boxscores_test = pd.read_sql('select * from team_advanced_boxscores', con)
advanced_game_ids_test = team_advanced_boxscores_test['GAME_ID'].unique()

missing_game_ids_test = np.setdiff1d(game_ids, advanced_game_ids_test)
print(len(missing_game_ids_test))
print(missing_game_ids_test)

73
['0020300086' '0020300453' '0020300778' '0020300833' '0020400002'
 '0020400003' '0020400020' '0020400023' '0020400029' '0020400048'
 '0020400190' '0020400554' '0020500010' '0020500017' '0020500019'
 '0020500020' '0020500021' '0020500022' '0020500028' '0020500030'
 '0020500032' '0020500038' '0020600103' '0020600111' '0020600119'
 '0020600127' '0020600128' '0020600141' '0020600637' '0020600667'
 '0020600670' '0020600672' '0020600673' '0020600682' '0020700927'
 '0020700976' '0020701078' '0020800104' '0020800859' '0020801063'
 '0021000552' '0021001153' '0021201036' '0021201037' '0021201202'
 '0021201214' '0021300797' '0021400627' '0021500139' '0021500141'
 '0021500151' '0021600004' '0021700109' '0021701174' '0021800152'
 '0021800566' '0021800911' '0022000447' '0022100041' '0022100215'
 '0022100461' '0022100463' '0022100534' '0022100692' '0022101047'
 '0022101216' '0040800104' '0040800124' '0040800152' '0040800162'
 '0040800165' '0041700232' '0041800233']


In [48]:
#pull missing game_ids data
final_missing_game_ids = []
for season in tqdm(range(2000, 2022)):
    missing_game_ids = update_team_advanced_boxscores(con, season, [])
    if missing_game_ids is not None:
        final_missing_game_ids.extend(missing_game_ids)
    
print('remaining missing game_ids: ', len(final_missing_game_ids))

  5%|▍         | 1/22 [00:00<00:12,  1.71it/s]

num_games_updated: 0
All team advanced boxscores up to date in season 2000-01


  9%|▉         | 2/22 [00:01<00:10,  1.82it/s]

num_games_updated: 0
All team advanced boxscores up to date in season 2001-02


 14%|█▎        | 3/22 [00:01<00:11,  1.59it/s]

...

num_games_updated: 0
All team advanced boxscores up to date in season 2020-21



progress:   0%|          | 0/1 [00:00<?, ?it/s]

num_games_updated: 1



100%|██████████| 22/22 [00:28<00:00,  1.28s/it]

remaining missing game_ids:  1


In [49]:
team_advanced_boxscores_test = pd.read_sql('select * from team_advanced_boxscores', con)
advanced_game_ids_test = team_advanced_boxscores_test['GAME_ID'].unique()

missing_game_ids_test = np.setdiff1d(game_ids, advanced_game_ids_test)
print(len(missing_game_ids_test))
print(missing_game_ids_test)

2
['0020300778' '0021201214']


In [43]:
team_advanced_boxscores[team_advanced_boxscores['GAME_ID']=='0021201214']

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE


In [55]:
team_advanced_boxscores_df = pd.read_sql('select * from team_advanced_boxscores', con)
team_advanced_boxscores_df

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022100002,1610612747,Lakers,LAL,Los Angeles,240:00,98.0,101.8,106.0,107.1,...,16.1,0.553,0.551,1.0,0.198,115.28,112.5,93.75,112.0,0.422
1,0022100002,1610612744,Warriors,GSW,Golden State,240:00,106.0,107.1,98.0,101.8,...,15.0,0.516,0.570,1.0,0.200,115.28,112.5,93.75,113.0,0.578
2,0022100001,1610612749,Bucks,MIL,Milwaukee,240:00,117.7,124.5,101.8,102.0,...,7.8,0.538,0.562,1.0,0.194,105.02,102.0,85.00,102.0,0.593
3,0022100001,1610612751,Nets,BKN,Brooklyn,240:00,101.8,102.0,117.7,124.5,...,12.7,0.542,0.552,1.0,0.200,105.02,102.0,85.00,102.0,0.407
4,0022100003,1610612754,Pacers,IND,Indiana,240:00,111.4,114.0,107.1,116.0,...,15.9,0.561,0.607,1.0,0.199,112.22,106.5,88.75,107.0,0.520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56301,0022100534,1610612746,Clippers,LAC,LA,240:00,112.1,113.7,116.6,118.4,...,13.7,0.583,0.625,1.0,0.199,97.94,96.5,80.42,95.0,0.500
56302,0022100463,1610612759,Spurs,SAS,San Antonio,240:00,109.8,113.7,90.4,91.1,...,11.8,0.491,0.497,1.0,0.197,103.72,101.5,84.58,102.0,0.642
56303,0022100463,1610612746,Clippers,LAC,LA,240:00,90.4,91.1,109.8,113.7,...,14.9,0.443,0.480,1.0,0.199,103.72,101.5,84.58,101.0,0.358
56304,0042100405,1610612744,Warriors,GSW,Golden State,240:00,106.6,107.2,95.3,95.9,...,7.2,0.517,0.550,1.0,0.197,98.12,97.5,81.25,97.0,0.563


In [56]:
#team_basic_boxscores_df.to_csv('team_basic_boxscores.csv', mode='w+')
team_advanced_boxscores_df.to_csv('team_advanced_boxscores.csv', mode='w+')

In [62]:
team_boxscores_df = team_basic_boxscores_df.merge(team_advanced_boxscores_df, how='inner', on=['GAME_ID', 'TEAM_ID'])
team_boxscores_df.columns

Index(['SEASON', 'TEAM_ID', 'TEAM_ABBREVIATION_x', 'TEAM_NAME_x', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN_x', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'TEAM_NAME_y',
       'TEAM_ABBREVIATION_y', 'TEAM_CITY', 'MIN_y', 'E_OFF_RATING',
       'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING',
       'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT',
       'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT',
       'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE'],
      dtype='object')